In [13]:
import pandas as pd
import os
MAXSTEP = 3032
MODELNAMES = [
    "18_normalised",
    "19_unnormalised",
    ]
VOCABS = {
"18_normalised": "vocab_300.json",
"19_unnormalised": "vocab_300_with_numbers.json"
}
COLS = {
"18_normalised": "human_transcript",
"19_unnormalised": "unnormalized_transcript"
}
CHECKPOINTS = [
    "checkpoint-757",
    "checkpoint-1514",  
    "checkpoint-2271", 
    "checkpoint-3028", 
]
dev = pd.read_csv("17_dev.csv")
test = pd.read_csv("17_test.csv")

from datasets import load_metric
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

results = list()
for modelname in MODELNAMES:
    vocabulary_file = VOCABS.get(modelname)
    print("This will prepare the following vocabulary:", vocabulary_file)#os.system(f"cp {vocabulary_file} vocab.json")
    for i, checkpoint in enumerate(CHECKPOINTS):
        steps = (i+1) * 757
        epochs = steps * 8 / MAXSTEP
        checkpoint_path = os.path.join(modelname, checkpoint)
        
        for df_name, df in zip(["dev", "test"], [dev, test]):
            col = COLS.get(modelname)
            references = df[col]
            SETUP =  dict(
                        modelname=modelname,
                        vocabulary_file=vocabulary_file,
                        checkpoint=checkpoint,
                        checkpoint_path=checkpoint_path,
                        df_name=df_name,
                        col=col,
                        steps=steps,
                        epochs=epochs
                        )
            print(SETUP)




This will prepare the following vocabulary: vocab_300.json
{'modelname': '18_normalised', 'vocabulary_file': 'vocab_300.json', 'checkpoint': 'checkpoint-757', 'checkpoint_path': '18_normalised/checkpoint-757', 'df_name': 'dev', 'col': 'human_transcript', 'steps': 757, 'epochs': 1.9973614775725594}
{'modelname': '18_normalised', 'vocabulary_file': 'vocab_300.json', 'checkpoint': 'checkpoint-757', 'checkpoint_path': '18_normalised/checkpoint-757', 'df_name': 'test', 'col': 'human_transcript', 'steps': 757, 'epochs': 1.9973614775725594}
{'modelname': '18_normalised', 'vocabulary_file': 'vocab_300.json', 'checkpoint': 'checkpoint-1514', 'checkpoint_path': '18_normalised/checkpoint-1514', 'df_name': 'dev', 'col': 'human_transcript', 'steps': 1514, 'epochs': 3.994722955145119}
{'modelname': '18_normalised', 'vocabulary_file': 'vocab_300.json', 'checkpoint': 'checkpoint-1514', 'checkpoint_path': '18_normalised/checkpoint-1514', 'df_name': 'test', 'col': 'human_transcript', 'steps': 1514, 'epo

checkpoint-1514  checkpoint-2271  checkpoint-3028  checkpoint-757  runs
